In [1]:
import numpy as np
import pandas as pd 
import requests 
from scipy import stats 
import math

In [2]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

df = pd.read_csv('S&P500.csv')
df.head()

,Symbol
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
Companies = {}  # stores company names along with mkt cap and latest price
Companies['Name'] = []
Companies['CMP'] = []
Companies['1 year change'] = []


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
batches  = list(chunks(df['Symbol'], 100)) # 

for i in range(len(batches)):
    batch = (','.join(batches[i]))
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for company in batch.split(','):
        Companies['Name'].append(company)
        Companies['CMP'].append(data[company]['quote']['latestPrice'])
        Companies['1 year change'].append(data[company]['stats']['year1ChangePercent'])
        

df = pd.DataFrame(Companies)
df

,Name,CMP,1 year change
0,A,125.83,-0.172606
1,AAL,14.60,-0.387994
2,AAP,186.10,-0.085574
3,AAPL,145.93,0.073447
4,ABBV,154.85,0.424133
...,...,...,...
497,YUM,116.75,-0.007243
498,ZBH,112.71,-0.340551
499,ZBRA,312.37,-0.413893
500,ZION,54.52,-0.038818


In [4]:
df = df.sort_values(by = '1 year change' , ascending = False)
df

,Name,CMP,1 year change
148,DVN,54.08,0.954768
354,OXY,59.23,0.767973
296,MCK,330.64,0.705171
40,APA,37.74,0.667211
90,CF,87.12,0.657666
...,...,...,...
236,ILMN,201.73,-0.617813
84,CCL,11.15,-0.644133
328,NFLX,193.09,-0.665110
207,GPS,9.51,-0.737357


In [5]:
top50 = df.head(50)
top50.reset_index(inplace = True)

In [6]:
top50.head(5)

,index,Name,CMP,1 year change
0,148,DVN,54.08,0.954768
1,354,OXY,59.23,0.767973
2,296,MCK,330.64,0.705171
3,40,APA,37.74,0.667211
4,90,CF,87.12,0.657666


In [7]:
portfolio_size = 100000

In [8]:
# shares to buy 
# allocating equal amount for each share

# Shares for each company
amount = portfolio_size/len(top50['Name'])

pd.set_option('mode.chained_assignment', None)
top50['Shares to buy'] = 0
for i in range(len(top50['Name'])):
    top50['Shares to buy'][i] = math.floor(amount/top50['CMP'][i])
    
top50

,index,Name,CMP,1 year change,Shares to buy
0,148,DVN,54.08,0.954768,36
1,354,OXY,59.23,0.767973,33
2,296,MCK,330.64,0.705171,6
3,40,APA,37.74,0.667211,52
4,90,CF,87.12,0.657666,22
5,139,DLTR,165.03,0.620881,12
6,315,MRO,22.12,0.597693,90
7,119,CTRA,26.39,0.584446,75
8,269,KSU,308.07,0.530676,6
9,109,COP,92.47,0.529481,21


In [ ]:
#filtering high quality momentum stocks -> steady returns
#low quality momtum stocks -> sudden jump
df1 = pd.read_csv('S&P500.csv')

Companies = {}  # stores company names along with mkt cap and latest price
Companies['Name'] = []
Companies['CMP'] = []
Companies['6 month change'] = []
Companies['6 month %ile'] = []
Companies['1 year change'] = []
Companies['1 year %ile'] = []
Companies['2 year change'] = []
Companies['2 year %ile'] = []
Companies['momentum score'] = []

batches  = list(chunks(df1['Symbol'], 100)) 

for i in range(len(batches)):
    batch = (','.join(batches[i]))
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for company in batch.split(','):
        Companies['Name'].append(company)
        Companies['CMP'].append(data[company]['quote']['latestPrice'])
        Companies['1 year change'].append(data[company]['stats']['year1ChangePercent'])
        Companies['2 year change'].append(data[company]['stats']['year2ChangePercent'])
        Companies['6 month change'].append(data[company]['stats']['month6ChangePercent'])
        Companies['6 month %ile'].append(0)
        Companies['1 year %ile'].append(0)
        Companies['2 year %ile'].append(0)
        Companies['momentum score'].append(0)

df1 = pd.DataFrame(Companies)
df1

In [ ]:
for i in range(len(df1['Name'])):
    df1['6 month %ile'][i] = stats.percentileofscore(df1['6 month change'],df1['6 month change'][i])/100
    df1['1 year %ile'][i] = stats.percentileofscore(df1['1 year change'],df1['1 year change'][i])/100
    df1['2 year %ile'][i] = stats.percentileofscore(df1['2 year change'],df1['2 year change'][i])/100
    df1['momentum score'][i] = (df1['6 month %ile'][i]+df1['1 year %ile'][i]+df1['2 year %ile'][i])/3

df1.head()

In [ ]:
# now selecting top 50 based on momentum score
df1 = df1.sort_values(by = 'momentum score' , ascending = False)
top50 = df1.head(50)
top50.reset_index(inplace = True)
top50.head()

In [ ]:
portfolio_size = 100000
amount = portfolio_size/50

pd.set_option('mode.chained_assignment', None)
top50['Shares to buy'] = 0
for i in range(len(top50['Name'])):
    top50['Shares to buy'][i] = math.floor(amount/top50['CMP'][i])
    
top50